- GLUE benchmark에 포함된 여러가지 task를 파악하고, 상황에 맞는 것을 취사선택할 수 있다.
- 필요한 Custom dataset을 직접 만들고, 이를 훈련시킬 수 있다.
- 더 나아가, Huggingface framework 사용법을 익히고, 이를 편하게 이용할 수 있다.

1. GLUE dataset과 Huggingface
2. 커스텀 프로젝트 제작 (1) Dataset
3. 커스텀 프로젝트 제작 (2) Tokenizer와 Model
4. 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test

GLUE benchmark에서 각 태스크에 사용된 metric은 어떠한 것들이 있었나요?  
Accuracy, F1-score, Pearson-Spearman Correlation, MCC(Matthew's Correlation Coefficient) 등이 사용되었습니다.

### Huggingface transformers 설치 및 환경 구성
$ python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

위의 코드가 잘 실행되지 않을 경우 터미널을 열고 아래 명령어를 입력하여 환경을 구성합니다.
$ pip uninstall transformers -y  

$ pip install transformers  

$ mkdir -p transformers

### GLUE의 'mrpc' task를 나만의 커스텀 프로젝트로 구성해서 해결해봅시다
MRPC(The Microsoft Research Paraphrase Corpus)는 주어진 두 문장이 동일한 의미를 가지고 있는지 여부를 분류하는 작업에 사용됩니다. Label은 0(not equivalent, 상이함)과 1(equivalent, 동일함) 두 가지로 나뉩니다.

# 커스텀 프로젝트 제작 (1) Datasets
## Huggingface dataset에서 불러오기

In [1]:
import datasets
from datasets import load_dataset

huggingface_mrpc_dataset = load_dataset('glue', 'mrpc')
print(huggingface_mrpc_dataset)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


huggingface mrpc dataset을 확인해보면 위와 같이 구성되어 있습니다.

Dataset dictionary안에 train dataset, validation dataset, test dataset으로 구성되어 있고 각 Dataset은 ‘sentence1’, ‘sentence2’, ‘label’, ‘idx’(인덱스)로 구성되어 있습니다. 해당 내용처럼 Huggingface datasets를 사용하면 손쉽게 모델의 input으로 사용할 수 있다는 장점이 있습니다.

In [2]:
# Train datasets의 각 컬럼에 해당하는 요소들
train = huggingface_mrpc_dataset['train']
cols = train.column_names
cols

['sentence1', 'sentence2', 'label', 'idx']

In [3]:
# 제대로 짝을 지어있는지 확인해보자
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

sentence1 : Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
sentence2 : Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
label : 1
idx : 0


sentence1 : Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
sentence2 : Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
label : 0
idx : 1


sentence1 : They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .
sentence2 : On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .
label : 1
idx : 2


sentence1 : Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .
sentence2 : Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .
label : 0

1. Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .

2. Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .

두 문장의 순서만 도치되었을 뿐, 의미가 같기에 label : 1로 동일한 문장이라 평가할 수 있겠네요!

## 커스텀 데이터셋 만들기
Huggingface에 원하는 데이터셋이 없다면 어떡하죠? 걱정마세요, Huggingface datasets API를 활용하면 데이터셋을 직접 만들 수도 있고, 업로드까지도 가능합니다!

이번에는 MRPC 데이터셋을 직접 만들어보도록 하겠습니다. GLUE 데이터셋은 홈페이지에서도 원본을 다운로드할 수도 있지만, tensorflow_datasets에서 불러와 Huggingface dataset에 맞게 가공해봅시다.

In [ ]:
import tensorflow_datasets as tfds
from datasets import Dataset

tf_dataset, tf_dataset_info = tfds.load('glue/mrpc', with_info=True)
tf_dataset

In [ ]:
examples = tf_dataset['train'].take(5)

for example in examples:
    for col in cols:
        print(col, ":", example[col])
    print('\n')

Huggingface dataset과의 큰 차이는 없어보입니다. 다만 데이터가 tf.Tensor형태로 묶여있다는 차이점을 확인할 수 있습니다.

Huggingface dataset가 이중 딕셔너리 내부에 데이터를 리스트 형태로 담았기에, 이와 같은 방식으로 tf dataset을 재구성합니다.

In [ ]:
# Tensorflow dataset 구조를 python dict 형식으로 변경
# Dataset이 train, validation, test로 나뉘도록 구성
train_dataset = tfds.as_dataframe(tf_dataset['train'], tf_dataset_info)
val_dataset = tfds.as_dataframe(tf_dataset['validation'], tf_dataset_info)
test_dataset = tfds.as_dataframe(tf_dataset['test'], tf_dataset_info)

# dataframe 데이터를 dict 내부에 list로 변경
train_dataset = train_dataset.to_dict('list')
val_dataset = val_dataset.to_dict('list')
test_dataset = test_dataset.to_dict('list')

# Huggingface dataset
tf_train_dataset = Dataset.from_dict(train_dataset)
tf_val_dataset = Dataset.from_dict(val_dataset)
tf_test_dataset = Dataset.from_dict(test_dataset)

# 커스텀 프로젝트 제작 (2) Tokenizer와 Model
## Huggingface Auto Classes를 이용하는 방법
Huggingface에서 Auto Model과 이에 상응하는 tokenizer을 불러오겠습니다.

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)

Huggingface의 경우 AutoTokenizer, AutoModel기능을 지원합니다.

AutoTokenizer와 AutoModel은 Huggingface에서 지원하는 Auto Class입니다.

Auto class는 from_pretrained 메소드를 이용해 pretrained model의 경로 혹은 이름만 안다면 자동으로 생성하는 방법입니다.

즉 bert를 사용할때 BertTokenizer, RoBERTa를 사용할때 RobertaTokenizer를 사용하게 되는데 AutoTokenizer를 이용하면 자동으로 BERT모델은 BERT로 RoBERTa모델은 RoBERTa로 바꿔줍니다.

model의 경우 AutoModel을 그대로 사용하기보다 특정 task를 지정하는 방식인 AutoModelForSequenceClassification을 사용하는걸 권장드립니다.

Auto class는 다양한 모델에 자동으로 맞출 수 있기 떄문에 특정 task와 dataset이 주어져있는 경우 모델을 다양하게 넣어 실험할 수 있습니다.

그렇기에 Auto class를 유용하게 활용하는 것을 추천합니다.

Tokenizer와 Model을 만들었다면 이제 토크나이징하는 방법을 알아보도록 하겠습니다.

토크나이징은 transform이라는 함수를 만들고 이전에 만들어두었던 Tokenizer를 사용하는데 이때 dataset의 형태를 확인하고 바꿀 대상을 지정해야 합니다.

mrpc의 경우 sentence1, sentence2가 토크나이징할 대상이므로 data[’sentence1’], data[’sentence2’]로 인덱싱해서 지정합니다.

truncation은 특정 문장이 길어 모델을 다루기 힘들어 질 수 있으므로 짧게 자르는 것을 의미합니다.

return_token_type_ids는 문장이 한개이상일 때 나뉘는걸 보여줍니다. (해당 내용은 task에 필요없으므로 제거합니다)

예시까지 확인하면 다음과 같이 확인할 수 있습니다.

huggingface_mrpc_dataset 의 train dataset 중 5개의 샘플을 불러와 토크나이저를 적용해보고, 그 결과를 출력해볼까요?

In [ ]:
def transform(data):
    return huggingface_tokenizer(
        data['sentence1'],
        data['sentence2'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

데이터셋을 한번에 토크나이징할때 자주 사용하는 기법은 map입니다.

map을 사용하게 되면 Data dictionary에 있는 모든 데이터들이 빠르게 적용시킬 수 있습니다.

우리는 map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True로 주어야 합니다.

In [ ]:
hf_dataset = huggingface_mrpc_dataset.map(transform, batched=True)

# train & validation & test split
hf_train_dataset = hf_dataset['train']
hf_val_dataset = hf_dataset['validation']
hf_test_dataset = hf_dataset['test']

이전 스텝에서 tfds의 MRPC로 만든 커스텀 데이터셋의 경우 그대로 적용할 경우 오류가 발생합니다. 어떤 오류가 발생하는지 확인해볼까요?

In [ ]:
# Q. tf_train_dataset에 transform 함수를 매핑해봅시다. 어떤 오류가 발생하나요?
# [[YOUR CODE]]
tf_train_dataset.map(transform, batched=True)

보아하니 데이터 타입이 맞지 않아 발생하는 오류로 보입니다. 이 경우 기존 transform 함수가 sentence를 불러올 때 디코딩을 해주면 해결됩니다.

새로운 함수 transform_df 를 아래와 같이 정의하고 커스텀 데이터셋에 적용해보겠습니다.

In [ ]:
def transform_tf(batch):
    sentence1 = [s.decode('utf-8') for s in batch['sentence1']]
    sentence2 = [s.decode('utf-8') for s in batch['sentence2']]
    return huggingface_tokenizer(
        sentence1,
        sentence2,
        truncation=True,
        padding='max_length',
        return_token_type_ids=False,
    )

# 토큰화 및 패딩을 적용
tf_train_dataset = tf_train_dataset.map(transform_tf, batched=True)
tf_val_dataset = tf_val_dataset.map(transform_tf, batched=True)
tf_test_dataset = tf_test_dataset.map(transform_tf, batched=True)

# 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test
## Trainer를 활용한 학습
이제 다 왔습니다! Huggingface의 Trainer을 활용해 학습을 진행해보도록 하겠습니다.

Trainer를 사용하기 위해서는 TrainingArguments를 통해 학습 관련 설정을 미리 지정해야 합니다.

In [ ]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 16,   # 각 device 당 batch size
    per_device_eval_batch_size = 16,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

아래에서 생성하게 될 Trainer의 인자로 넘겨주어야 할 것 중에 compute_metrics 메소드가 있습니다.

이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정해 두는 것입니다.

MRPC 데이터셋은 binary classification에 해당하겠죠?

In [ ]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Trainer에 model, arguments, train_dataset, eval_dataset, compute_metrics를 넣고 train을 진행합니다.

In [ ]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

마지막으로 test 데이터셋으로 평가를 합니다.

In [ ]:
trainer.evaluate(hf_test_dataset)

방금 학습시킨 모델은 Huggingface dataset으로 학습시켰는데요, 앞에서 우리가 만든 커스텀 데이터셋 (tf_train_dataset, tf_val_dataset, tf_test_dataset)으로도 학습을 진행해보고, 결과를 비교해봅시다.

In [ ]:
#메모리를 비워줍니다.
del huggingface_model

In [ ]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=tf_train_dataset,    # training dataset
    eval_dataset=tf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
trainer.evaluate(tf_test_dataset)

In [ ]:
#메모리를 비워줍니다.
del huggingface_model